# Layers without Parameters

In [10]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
class CenterLayer(nn.Module):
    def __init__(self):
        super(CenterLayer, self).__init__()
        
    def forward(self, data):
        return data - data.mean()

In [6]:
x = torch.randn(2,3)
layer = CenterLayer()
y = layer(x)
x, y, y.mean()

(tensor([[-1.1141,  0.3150,  0.3315],
         [ 1.9382,  0.7611, -0.5618]]),
 tensor([[-1.3924,  0.0367,  0.0532],
         [ 1.6599,  0.4828, -0.8401]]),
 tensor(9.9341e-09))

We can also use it to construct more complex models.

In [8]:
net = nn.Sequential(nn.Linear(3,2), CenterLayer())
net(x), net(x).mean()

(tensor([[-0.6968,  0.8588],
         [-0.4068,  0.2448]], grad_fn=<SubBackward0>),
 tensor(-1.1176e-08, grad_fn=<MeanBackward0>))

# Layers with Parameters

For instance, this way we don’t need to write custom serialization routines for each new custom layer. For instance, we can use the member variable params of the ParameterDict type that comes with the Module class. It is a dictionary that maps string type parameter names to model parameters in the Parameter type. We can create a Parameter instance from ParameterDict via the get function.

In [14]:
param = nn.ParameterDict()
param.update({'param2': nn.Parameter(Variable(torch.arange(6)).reshape(2,3).float())})
param

ParameterDict(  (param2): Parameter containing: [torch.FloatTensor of size 2x3])

In [16]:
param.param2.data

tensor([[0., 1., 2.],
        [3., 4., 5.]])

In [34]:
class MyDense(nn.Module):
    def __init__(self, inchannel, outchannel):
        super(MyDense, self).__init__()
        self.weight = Variable(torch.randn(inchannel,outchannel))
        self.bias = Variable(torch.ones(outchannel,))
        
    def forward(self, data):
#         print(torch.mm(data, self.weight).shape, self.bias.shape)
        return torch.mm(data, self.weight) + self.bias
        

In [35]:
x = torch.arange(6).reshape(2,3).float()
net = MyDense(3,2)
out = net(x)
x, out

(tensor([[0., 1., 2.],
         [3., 4., 5.]]),
 tensor([[ 1.3661,  3.5697],
         [ 1.2357, 10.7727]]))

In [36]:
net.parameters

<bound method Module.parameters of MyDense()>

In [37]:
net.state_dict()

OrderedDict()